In [35]:
from arcexplore import *
import polars as pl

# Creation of Input/Output complete Dataframe

In [36]:
rows = [(k,d,id,tt,i,it.input.shape, it.output.shape) for k,d,id,tt,i,it in ARC_DATA.data_iter()]
complete_scheme = ("Source", "Set Type","ID","Task Type","Index","Input Shape","Output Shape")
complete_frame = pl.DataFrame(rows,
                              schema = complete_scheme,
                              orient="row")
display(complete_frame)

Source,Set Type,ID,Task Type,Index,Input Shape,Output Shape
str,str,str,str,i64,list[i64],list[i64]
"""ARC-AGI-1""","""training""","""007bbfb7""","""train""",0,"[3, 3]","[9, 9]"
"""ARC-AGI-1""","""training""","""007bbfb7""","""train""",1,"[3, 3]","[9, 9]"
"""ARC-AGI-1""","""training""","""007bbfb7""","""train""",2,"[3, 3]","[9, 9]"
"""ARC-AGI-1""","""training""","""007bbfb7""","""train""",3,"[3, 3]","[9, 9]"
"""ARC-AGI-1""","""training""","""007bbfb7""","""train""",4,"[3, 3]","[9, 9]"
…,…,…,…,…,…,…
"""ARC-AGI-2""","""evaluation""","""faa9f03d""","""test""",1,"[12, 12]","[12, 12]"
"""ARC-AGI-2""","""evaluation""","""fc7cae8d""","""train""",0,"[22, 22]","[11, 15]"
"""ARC-AGI-2""","""evaluation""","""fc7cae8d""","""train""",1,"[22, 22]","[10, 10]"


# Counting Tasks

In [37]:
task_frame = complete_frame.drop(("Task Type","Index","Input Shape","Output Shape")).unique()
display(task_frame)

Source,Set Type,ID
str,str,str
"""ARC-AGI-2""","""training""","""7d18a6fb"""
"""ARC-AGI-1""","""training""","""68b16354"""
"""ARC-AGI-1""","""evaluation""","""af24b4cc"""
"""ARC-AGI-2""","""training""","""1e81d6f9"""
"""ARC-AGI-2""","""training""","""23581191"""
…,…,…
"""ARC-AGI-1""","""evaluation""","""e760a62e"""
"""ARC-AGI-1""","""training""","""6e19193c"""
"""ARC-AGI-2""","""training""","""5792cb4d"""


In [38]:
group_cols = ("Source", "Set Type")

for i in range(len(group_cols)):
    fr = task_frame.group_by(group_cols[:len(group_cols)-i]).len("COUNT")
    display(fr)

Source,Set Type,COUNT
str,str,u32
"""ARC-AGI-2""","""evaluation""",120
"""ARC-AGI-2""","""training""",1000
"""ARC-AGI-1""","""evaluation""",400
"""ARC-AGI-1""","""training""",400


Source,COUNT
str,u32
"""ARC-AGI-2""",1120
"""ARC-AGI-1""",800


# Counting Grid Pairs

In [40]:
grid_frame = complete_frame.drop(("Input Shape","Output Shape")).unique()
display(grid_frame)

Source,Set Type,ID,Task Type,Index
str,str,str,str,i64
"""ARC-AGI-2""","""training""","""a1570a43""","""train""",2
"""ARC-AGI-2""","""training""","""c3e719e8""","""test""",0
"""ARC-AGI-2""","""training""","""a834deea""","""train""",2
"""ARC-AGI-2""","""training""","""7df24a62""","""train""",0
"""ARC-AGI-2""","""training""","""1c02dbbe""","""train""",2
…,…,…,…,…
"""ARC-AGI-2""","""training""","""ac0a08a4""","""train""",1
"""ARC-AGI-2""","""evaluation""","""4c416de3""","""test""",0
"""ARC-AGI-2""","""evaluation""","""dbff022c""","""train""",1


In [42]:
group_cols = ("Source", "Set Type","ID","Task Type")
for i in range(len(group_cols)):
    fr = grid_frame.group_by(group_cols[:len(group_cols)-i]).len("COUNT")
    display(fr)

Source,Set Type,ID,Task Type,COUNT
str,str,str,str,u32
"""ARC-AGI-1""","""training""","""95990924""","""train""",3
"""ARC-AGI-2""","""training""","""d06dbe63""","""test""",1
"""ARC-AGI-1""","""evaluation""","""13713586""","""test""",1
"""ARC-AGI-2""","""evaluation""","""898e7135""","""train""",2
"""ARC-AGI-1""","""training""","""760b3cac""","""train""",3
…,…,…,…,…
"""ARC-AGI-2""","""training""","""c48954c1""","""train""",3
"""ARC-AGI-2""","""training""","""e99362f0""","""test""",1
"""ARC-AGI-2""","""training""","""264363fd""","""test""",1


Source,Set Type,ID,COUNT
str,str,str,u32
"""ARC-AGI-2""","""training""","""bbc9ae5d""",6
"""ARC-AGI-2""","""training""","""2dd70a9a""",4
"""ARC-AGI-1""","""training""","""d631b094""",5
"""ARC-AGI-2""","""training""","""21f83797""",3
"""ARC-AGI-2""","""training""","""7e0986d6""",3
…,…,…,…
"""ARC-AGI-1""","""evaluation""","""60a26a3e""",4
"""ARC-AGI-2""","""training""","""b25e450b""",4
"""ARC-AGI-2""","""training""","""e2092e0c""",4


Source,Set Type,COUNT
str,str,u32
"""ARC-AGI-1""","""training""",1718
"""ARC-AGI-1""","""evaluation""",1782
"""ARC-AGI-2""","""training""",4308
"""ARC-AGI-2""","""evaluation""",529


Source,COUNT
str,u32
"""ARC-AGI-2""",4837
"""ARC-AGI-1""",3500


# Counting Grid Cells